In [1]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
import numpy as np
dataset=pd.read_csv('/kaggle/input/en-fr-translation-dataset/en-fr.csv')
dataset.head()

2024-05-19 18:16:34.289171: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 18:16:34.289331: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 18:16:34.449474: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


,en,fr
0,Changing Lives | Changing Society | How It Wor...,Il a transformé notre vie | Il a transformé la...
1,Site map,Plan du site
2,Feedback,Rétroaction
3,Credits,Crédits
4,Français,English


In [2]:
len(dataset['en'][1])

8

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')
def encoder(text):
    input_ids = tokenizer.encode(text, return_tensors='tf')
    outputs = model(input_ids)
    cls_embedding = outputs.last_hidden_state[:, 0, :]
    angle_rates = 1 / np.power(10000, (2 * (768 // 2)) / np.float32(512))
    pos = np.arange(50)[:, np.newaxis]
    angle_rads = pos * angle_rates
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)
    cls_embedding += pos_encoding[:, :50, :]
    return cls_embedding
    

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [4]:
encoder("Some weights of the ")

<tf.Tensor: shape=(1, 50, 768), dtype=float32, numpy=
array([[[-0.4258768 ,  0.3177908 , -0.1517996 , ..., -0.5867447 ,
          0.03612775,  0.47276962],
        [-0.42587578,  0.31779182, -0.1517986 , ..., -0.5867437 ,
          0.03612875,  0.47277063],
        [-0.4258748 ,  0.3177928 , -0.15179761, ..., -0.5867427 ,
          0.03612975,  0.4727716 ],
        ...,
        [-0.4258298 ,  0.3178378 , -0.1517526 , ..., -0.5866977 ,
          0.03617475,  0.47281662],
        [-0.42582878,  0.31783882, -0.15175161, ..., -0.58669674,
          0.03617575,  0.47281763],
        [-0.4258278 ,  0.3178398 , -0.15175061, ..., -0.58669573,
          0.03617675,  0.4728186 ]]], dtype=float32)>